In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 5184 rows, 5184 columns and 5184 nonzeros
Model has 13439520 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+00, 4e+07]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 5184 rows and 0 columns
Presolve time: 1.82s
Presolved: 0 rows, 5184 columns, 0 nonzeros
Presolved model has 13439520 quadratic objective terms
Ordering time: 0.04s

Barrier statistics:
 Free vars  : 3695
 AA' NZ     : 6.825e+06
 Factor NZ  : 6.828e+06 (roughly 60 MBytes of memory)
 Factor Ops : 1.682e+10 (roughly 3 seconds per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.38842032e+10 -2.33023634e+05  3.01e+05 4.57e+04  3.46e+06     8s
   1   4.42008053e+09 -3.20974002e+08  2.62e+04 3.98e+03  5.75e+05    12s
   2   3.70137647e+08 -2.13167708e+08  9.68

In [2]:
# x = GLS_Apr_weekday_PM.x; # sample matrix, each column is a link flow vector sample; number_of_links * K
# S = GLS_Apr_weekday_PM.S; # sample covariance matrix
# A = GLS_Apr_weekday_PM.A; # link_route incidence matrix
P = GLS_Apr_weekday_PM.P; # route_choice_probability_matrix
# Q = GLS_Apr_weekday_PM.Q; 
L = GLS_Apr_weekday_PM.L; # dimension of xi
number_of_routes = GLS_Apr_weekday_PM.number_of_routes;
number_of_links = GLS_Apr_weekday_PM.number_of_links;

In [3]:
# A = sparse(A);
P = sparse(P);
# Q = sparse(Q);

In [4]:
# K = size(x, 2)

# inv_S = inv(S)

# A_t = transpose(A)

# b = sum([A_t * inv_S * x[:, k] for k = 1:K])

In [5]:
using JuMP

### implement GLS method to estimate OD demand matrix

In [6]:
# model = Model(solver=IpoptSolver())

# @variable(model, xi[1:L] >= 0)

# # Set objective: (K/2) xi' * Q * xi - b' * xi
# obj = 0
# for i = 1:L
#     for j = 1:L
#         obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
#     end
# end
# for l = 1:L
#     obj += - b[l] * xi[l]
# end

# @NLobjective(model, Min, obj) 

# solve(model)

In [7]:
# xi_list = getvalue(xi)

xi_list = GLS_Apr_weekday_PM.xi_list

5184-element Array{Any,1}:
  772.458     
  173.096     
  854.929     
    3.13892   
    0.0450955 
  959.809     
 1042.28      
    0.173997  
 1752.54      
    0.288549  
    0.0565601 
  744.938     
 1973.57      
    ⋮         
    0.138578  
    1.98923e-5
    1.97759e-5
    1.99852e-5
    2.10235e-5
    1.74978e-5
    4.70417e-5
    3.28338e-5
    2.76389e-5
    0.337478  
    0.109441  
    0.065578  

In [ ]:
mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)

In [ ]:
getvalue(lam)

In [ ]:
getobjectivevalue(mGLSJulia)